In [1]:
from Bio import SearchIO
from Bio.SeqIO import *

from tst.domutil.util import *
from tst.utils_db import *


# os.environ["SEQlib"] = full("$DATA/sequence-data")

/export/people/zcbtfge/Documents/repos/test_envs/django_CATH/local/lib/python2.7/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [3]:
# reload(tst.domutil.util)
%load_ext autoreload
%autoreload 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
### Manually generated tmp.hmmpf and tmp.domtbl

### search_
wkdir = "/data/sequence-data/"
hmm_profile = "%s/tmp.hmmpf" % wkdir
hmm_domtbl = "%s/tmp.domtbl" % wkdir
# CATH_S35

# kwargs = {"format":"hmmer3-domtab"}
tbl_format = "hmmsearch3-domtab"

In [5]:
import os
os.environ["SEQlib"]

'/export/people/zcbtfge/data/SEQlib'

## import sequence database

In [5]:
print seqDB_curr.sequence_set.count()
print sequence.objects.count()

NameError: name 'seqDB_curr' is not defined

In [12]:
jdict = {
    'name':'CATH',
    'version':'4_1_0'}
seqDB_curr = verify_exist_entry(jdict,seqDB)


In [18]:
debug = 1
p_cathheader = re.compile('\|([0-9,a-z,A-Z]*)\/')

behave = "import sequence database"
seqdb_file = full("$SEQlib/cath-domain-seqs-S100.fa")

from time import time
t0 = time()
useFisrt = 1
failcount = 0

with open( seqdb_file ) as handle:
    iterator = FastaIO.SimpleFastaParser(handle)
    c = counter(iterator, per=1000)
with open( seqdb_file ) as handle:
    iterator = FastaIO.SimpleFastaParser(handle)

    with transaction.atomic():
        for values in iterator:
            try:
#             if 1:
                header,seq = values
#                 if c.i == 0:
#                     (h_parser, seqDB_curr) = seqheader_guess_parser(header)
#                     print "first entry pointing to %s" % seqDB_curr
#                     print "h_parser is",h_parser
                    
#                 jdict = h_parser( header )
                jdict = {}
                jdict['acc'] = p_cathheader.findall(header)[0]
                jdict["length"] = len(seq)
                jdict["seqDB"] = seqDB_curr
            
#                 if useFisrt:
#                     jdict["seqDB"] = seqDB_curr
#                 else:
#                     jdict["seqDB"] = seqDB.objects.get(**jdict["seqDB"])

                seqobjs = sequence.objects.filter(**jdict)
                if not seqobjs.exists():
                    seqobj = sequence(**jdict)
                    seqobj.save()
#                     print "created %s"%jdict['acc']
                else:
                    if seqobjs.count() > 1:
                        print jdict['acc']
#             try:
                pass
            except Exception as e:
                if debug: print >>sys.__stdout__,e,header
                c.fail(str(e),header)
                failcount += 1
            c.count()
        
#         print(values)
print "finshed %s from %s" % (behave, seqdb_file)

print '%d instances of %d failed' % (failcount,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = failcount/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % (failrate*100)

created 102mA00
0 of 80422
created 107lA00
created 108lA00
created 109lA00
created 110lA00
created 111lA00
created 112lA00
created 113lA00
created 114lA00
created 115lA00
created 118lA00
created 119lA00
created 120lA00
created 122lA00
created 123lA00
created 125lA00
created 126lA00
created 127lA00
created 128lA00
created 129lA00
created 12asA00
created 12caA00
created 12e8H01
created 12e8L01
created 12e8L02
created 130lA00
created 131lA00
created 132lA00
created 133lA00
created 134lA00
created 137lA00
created 138lA00
created 139lA00
created 140lA00
created 141lA00
created 142lA00
created 143lA00
created 144lA00
created 145lA00
created 146lA00
created 147lA00
created 149lA00
created 14gsA01
created 151lA00
created 152lA00
created 153lA00
created 155cA00
created 155lA00
created 156lA00
created 157lA00
created 158lA00
created 159lA00
created 160lA00
created 161lA00
created 162lA00
created 163lA00
created 164lA00
created 165lA00
created 166lA00
created 167lA00
created 168lA00
created 169lA

created 1akhB00
created 1akoA00
created 1akpA00
created 1akqA00
created 1akrA00
created 1aksA00
created 1aksB00
created 1aktA00
created 1akwA00
created 1akyA00
created 1al0B00
created 1al2100
created 1al3A01
created 1al3A02
created 1alaA03
created 1alcA00
created 1aldA00
created 1alhA00
created 1aliA00
created 1alkA00
created 1allA00
created 1allB00
created 1alo006
created 1alqA00
created 1aluA00
created 1alyA00
created 1am2A00
created 1am3000
created 1am4A00
created 1am4D00
created 1am5A01
created 1am5A02
created 1am7A00
created 1am7B00
created 1am7C00
created 1am9B00
created 1ameA00
created 1amhA01
created 1amiA02
created 1amiA04
created 1amkA00
created 1amlA00
created 1ammA01
created 1ammA02
created 1amoA02
created 1amuA02
created 1amuA03
created 1amuA04
created 1amuB01
created 1amxA00
created 1an0A00
created 1an0B00
created 1an1E02
created 1an2A00
created 1an4A00
created 1an5A00
created 1an5B00
created 1an8A01
created 1an8A02
created 1an9A02
created 1anbA01
created 1ancA01
created 

created 1bapA02
created 1barB00
created 1bazC00
created 1bb3A00
created 1bb4A00
created 1bb8A00
created 1bb9A00
created 1bbgA00
created 1bbhA00
created 1bbjA01
created 1bbjA02
created 1bbjB01
created 1bbjH01
created 1bbnA00
created 1bboA01
created 1bboA02
created 1bbpA00
created 1bbuA01
created 1bbuA02
created 1bbwA02
created 1bbxC00
created 1bbyA00
created 1bc0A03
created 1bc1A04
created 1bc3A01
created 1bc3A02
created 1bc3A03
created 1bc4A00
created 1bc6A00
created 1bc8C00
created 1bc9A01
created 1bccA01
created 1bccA02
created 1bccB01
created 1bccB02
created 1bccD01
created 1bccD02
created 1bccE01
created 1bccF00
created 1bccG00
created 1bccH00
created 1bccJ00
created 1bcgA00
created 1bch100
created 1bcj100
created 1bcoA01
created 1bcoA02
created 1bcpA00
created 1bcpB01
created 1bcpB02
created 1bcpC01
created 1bcpC02
created 1bcpD00
created 1bcpF00
created 1bctA00
created 1bcwA01
created 1bcxA00
created 1bczA01
created 1bd0A01
created 1bd0B02
created 1bd2D01
created 1bd2E01
created 

created 1by5A02
created 1by8A00
created 1bybA00
created 1byfA00
created 1bygA01
created 1bygA02
created 1byiA00
created 1bykA01
created 1bykA02
created 1bymA00
created 1byoA00
created 1bypA00
created 1byqA00
created 1byrA00
created 1byuA00
created 1bywA00
created 1bz4A00
created 1bz7A01
created 1bz7A02
created 1bz7B01
created 1bzhA00
created 1bzkA00
created 1bzoA00
created 1bzqK00
created 1bzzA00
created 1c00A00
created 1c01A00
created 1c02A00
created 1c05A01
created 1c05A02
created 1c07A00
created 1c09A00
created 1c0aA01
created 1c0aA02
created 1c0aA03
created 1c0eA00
created 1c0fA01
created 1c0gA01
created 1c0gA03
created 1c0gS00
created 1c0mA01
created 1c0nA01
created 1c0nA02
created 1c0pA01
created 1c0pA02
created 1c0tA03
created 1c0tB01
created 1c0tB02
created 1c0uA03
created 1c0wA01
created 1c12A01
created 1c12B01
created 1c12B02
created 1c16A01
created 1c16A02
created 1c17M00
created 1c1bA03
created 1c1cA03
created 1c1dA01
created 1c1dA02
created 1c1dB01
created 1c1eH01
created 

created 1co9A00
created 1coaI00
created 1cokA00
created 1colA00
created 1corA00
created 1cotA00
created 1couA00
created 1cov100
created 1cov200
created 1cov300
created 1cov400
created 1coyA01
created 1coyA02
created 1cozA00
created 1cp0A00
created 1cp2A00
created 1cp5A00
created 1cp9A01
created 1cp9A02
created 1cp9B01
created 1cp9B02
created 1cp9B03
created 1cpcA00
created 1cpcB00
created 1cpcL00
created 1cpdA01
created 1cpgA01
created 1cpiA00
created 1cpiB00
created 1cpmA00
created 1cpnA00
created 1cpoA00
created 1cpqA00
created 1cprA00
created 1cptA00
created 1cpuA01
created 1cpv000
created 1cpwA00
created 1cpyA01
created 1cpyA02
created 1cpzA00
created 1cq3A00
created 1cqaA00
created 1cqdA00
created 1cqgA00
created 1cqkA00
created 1cqmA00
created 1cqqA01
created 1cqqA02
created 1cqsA00
created 1cqwA00
created 1cqxA01
created 1cqxA02
created 1cqxA03
created 1cqzA01
created 1cqzB01
created 1cqzB02
created 1cr2A00
created 1cr5A01
created 1cr5B02
created 1cr6A02
created 1cr8A00
created 

...

created 4nrdE00
created 4nrdF00
created 4nreA01
created 4nrgB00
created 4nriB00
created 4nrjA02
created 4nrtA03
created 4nrvA02
created 4nrwA01
created 4nrxB01
created 4nrxD01
created 4nrzB01
created 4ns0A00
created 4ns1A00
created 4ns1B00
created 4ns1C00
created 4ns1D00
created 4ns2A00
created 4nslA01
created 4nslB02
created 4nsnA00
created 4nsnB00
created 4nsnC00
created 4nstA01
created 4nstA02
created 4nstC02
created 4nsvA01
created 4nsvA02
created 4nsyA02
created 4nt3A00
created 4nt4A01
created 4nt4A02
78000 of 80422
created 4nt6A01
created 4nt6A02
created 4nt8A00
created 4ntkA00
created 4ntlA02
created 4ntmA00
created 4ntmB00
created 4ntmC00
created 4ntmD00
created 4ntmE00
created 4ntmF00
created 4ntnA00
created 4ntnB00
created 4ntnC00
created 4ntnD00
created 4ntnE00
created 4ntnF00
created 4ntwB00
created 4ntwC00
created 4ntxB00
created 4ntyB00
created 4nu0A00
created 4nu5A01
created 4nu5A02
created 4nu6A01
created 4nu7C00
created 4nu8X01
created 4nuaA00
created 4nufA02
created 4

created 4or5A02
created 4or5F02
created 4or7A00
created 4orfA01
created 4orfA02
created 4orgA01
created 4orgC01
created 4orgE01
created 4orgH01
created 4oriA00
created 4orxA00
created 4orzA00
created 4orzB00
created 4orzC00
created 4os0A00
created 4os3B00
created 4os8B00
created 4ospD00
created 4osuH01
created 4otcA00
created 4otdA02
created 4oteA01
created 4oteA02
created 4oteB01
created 4otgA02
created 4otkA00
created 4otwA01
created 4otwA02
created 4otxH01
created 4otxI01
created 4otxL01
created 4ou0A00
created 4ou1A00
created 4ou4A00
created 4ou5A00
created 4oujA01
created 4oujA02
created 4oulA00
created 4oumA00
created 4ousA00
created 4ov0A00
created 4ov6F00
created 4ovdA01
created 4ovjA01
created 4ovjA02
created 4ovmA00
created 4ovmB00
created 4ovmC00
created 4ovmD00
created 4ovmE00
created 4ovmF00
created 4ovmG00
created 4ovmH00
created 4ovmI00
created 4ovmJ00
created 4ovwA00
created 4ovwB00
created 4ow1A00
created 4ow4A00
created 4owrA00
created 4owrC00
created 4ox2A01
created 

created 4q31E01
created 4q31E02
created 4q31F01
created 4q31F02
created 4q31G01
created 4q31G02
created 4q31H01
created 4q31H02
created 4q32A00
created 4q3hA00
created 4q3rA00
created 4q3rB00
created 4q40D00
created 4q4hA01
created 4q4hB01
created 4q4jA01
created 4q4lA01
created 4q4lA02
created 4q4lA03
created 4q4w100
created 4q4w200
created 4q4w300
created 4q5eC00
created 4q5kA01
created 4q5kA02
created 4q5tA01
created 4q5tA02
created 4q60A01
created 4q61B00
created 4q66F00
created 4q67A00
created 4q68A01
created 4q68A02
created 4q6aA00
created 4q6bA01
created 4q6bA02
created 4q6gA02
created 4q6iA01
created 4q6iB01
created 4q6kA01
created 4q6kA02
created 4q6kB01
created 4q6kB02
created 4q6kC01
created 4q6kC02
created 4q6kD01
created 4q6kD02
created 4q6rA02
created 4q6rA03
created 4q6rB03
created 4q75A01
created 4q75A02
created 4q75B01
created 4q75B02
created 4q76A01
created 4q76A02
created 4q76B02
created 4q7fA01
created 4q7fA02
created 4q7gA00
created 4q7iA00
created 4q7nA01
created 

created 4tloA02
created 4tlpA00
created 4tmaE01
created 4tmkA00
created 4tn8A00
created 4tncA01
created 4tncA02
created 4tnmA00
created 4tnoA01
created 4tntA00
created 4tnwF01
created 4tnwK01
created 4to8B00
created 4topB01
created 4toqA00
created 4toyH01
created 4toyH02
created 4toyL01
created 4toyL02
created 4tpiI00
created 4tpkB00
created 4tpoA00
created 4tptB02
created 4tpwB00
created 4tq2A00
created 4tqdA02
created 4tqgA00
created 4tqoA00
created 4tqoI00
created 4tqtA01
created 4tqtB01
created 4tqtC01
created 4tqtD01
created 4tqtE01
created 4tqtF01
created 4tr0A00
created 4tr0B00
created 4tr1A00
created 4tr1B00
created 4tr5A00
created 4tr7B01
created 4tr7B02
created 4tr7B03
created 4tr8B01
created 4tr8B02
created 4tr8B03
created 4trrG00
created 4ts1A02
created 4tsfI00
created 4tskA01
created 4tsvA00
created 4tt3G02
created 4tt3H00
created 4tt6A00
created 4ttbA00
created 4ttrA00
created 4tu1D00
created 4tu7A01
created 4tu7B01
created 4tumA00
created 4tuuA01
created 4tv4C00
created 

created 4wkmA02
created 4wkqA02
created 4wksA00
created 4wksC01
created 4wksC02
created 4wksC03
created 4wlbA00
created 4wlhA01
created 4wlhA02
created 4wlhB02
created 4wljA02
created 4wm9A00
created 4wmcD00
created 4wmcE00
created 4wmcF00
created 4wmhA00
created 4wniA02
created 4wnlD00
created 4wnqB01
created 4wo4A02
created 4wokA01
created 4wokA02
created 4wp0A02
created 4wp0B02
created 4wp0C02
created 4wp0D02
created 4wr2A00
created 4wrrA01
created 4wshA00
created 4wsjB01
created 4wskC01
created 4wsoA00
created 4wtoA02
created 4wumA01
created 4wumA02
created 4wunA01
created 4wuoA00
created 4wvlA02
created 4wwhA01
created 4wwhA02
created 4wwhB01
created 4wwhB02
created 4wwlA02
created 4wwnA01
created 4wwoA01
created 4wxtA00
created 4wz8B02
created 4wz8B03
created 4wz8C01
created 4wzaF00
created 4wzzA01
created 4wzzA02
created 4x0mA01
created 4x23A00
created 4x2rA00
created 4x3gA01
created 4x7dC00
created 4x7fD00
created 4x84A02
created 4x84B01
created 4x9cD00
created 4x9sA00
created 

In [94]:
transaction.get_autocommit()

True

In [85]:
# jdict['acc']
c = counter([],INF=True)
acclst = []
with open( seqdb_file ) as handle:
    iterator = FastaIO.SimpleFastaParser(handle)
    for x,y in iterator:
        c.count()
        acclst.append(p_cathheader.findall(x)[0])

0 of -1
100 of -1
200 of -1
300 of -1
400 of -1
500 of -1
600 of -1
700 of -1
800 of -1
900 of -1
1000 of -1
1100 of -1
1200 of -1
1300 of -1
1400 of -1
1500 of -1
1600 of -1
1700 of -1
1800 of -1
1900 of -1
2000 of -1
2100 of -1
2200 of -1
2300 of -1
2400 of -1
2500 of -1
2600 of -1
2700 of -1
2800 of -1
2900 of -1
3000 of -1
3100 of -1
3200 of -1
3300 of -1
3400 of -1
3500 of -1
3600 of -1
3700 of -1
3800 of -1
3900 of -1
4000 of -1
4100 of -1
4200 of -1
4300 of -1
4400 of -1
4500 of -1
4600 of -1
4700 of -1
4800 of -1
4900 of -1
5000 of -1
5100 of -1
5200 of -1
5300 of -1
5400 of -1
5500 of -1
5600 of -1
5700 of -1
5800 of -1
5900 of -1
6000 of -1
6100 of -1
6200 of -1
6300 of -1
6400 of -1
6500 of -1
6600 of -1
6700 of -1
6800 of -1
6900 of -1
7000 of -1
7100 of -1
7200 of -1
7300 of -1
7400 of -1
7500 of -1
7600 of -1
7700 of -1
7800 of -1
7900 of -1
8000 of -1
8100 of -1
8200 of -1
8300 of -1
8400 of -1
8500 of -1
8600 of -1
8700 of -1
8800 of -1
8900 of -1
9000 of -1
9100 of -1


75900 of -1
76000 of -1
76100 of -1
76200 of -1
76300 of -1
76400 of -1
76500 of -1
76600 of -1
76700 of -1
76800 of -1
76900 of -1
77000 of -1
77100 of -1
77200 of -1
77300 of -1
77400 of -1
77500 of -1
77600 of -1
77700 of -1
77800 of -1
77900 of -1
78000 of -1
78100 of -1
78200 of -1
78300 of -1
78400 of -1
78500 of -1
78600 of -1
78700 of -1
78800 of -1
78900 of -1
79000 of -1
79100 of -1
79200 of -1
79300 of -1
79400 of -1
79500 of -1
79600 of -1
79700 of -1
79800 of -1
79900 of -1
80000 of -1
80100 of -1
80200 of -1
80300 of -1
80400 of -1


In [87]:
with open( seqdb_file ) as handle:
    acclst2  = p_cathheader.findall(handle.read()) 

In [92]:
print sequence.objects.count()

80407


## Importing HMM profiles from .lib file

In [10]:
%load_ext autoreload
%autoreload 2


In [11]:
seqheader_parse_cath(p_header.findall(hmm_text)[0]).get("acc")

NameError: name 'hmm_text' is not defined

In [4]:
from time import time
from tst.domutil.util import *
import re
p_header = re.compile("NAME.*?\n")
# from copy import copy
hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))
behave = "importing HMM profiles"


t0 = time()
lst = parse_hmmlib( hmmlib_file )
c = counter( lst)
lst = parse_hmmlib( hmmlib_file )
failcount = 0

try:
    with transaction.atomic():
        for hmm_text in lst:
            c.count()
            acc = seqheader_parse_cath(
                p_header.findall(hmm_text)[0]
            ).get("acc")
#             acc = next(header_gen)
            try:
                cnode = domain.objects.get(domain_id = acc).classification
                if cnode.hmmprofile_set.exists():
                    pass
                else:
                    hmm = HMMprofile(
                                cath_node_id = cnode.id,
                                text = hmm_text,
                                length = int(p_hmmlen.findall(hmm_text)[0])
                                )
                    hmm.save()
            except Exception as e:
                print "failed for %s for Exception:%s" %( acc, e)
                failcount += 1
except Exception as e:
    print "early stop due to %s" % e

print "finshed %s from %s" % (behave, hmmlib_file)

print '%d instances of %d failed' % (failcount,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = failcount/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate

 0 of 21155
failed for 1a7cA02 for Exception:get() returned more than one domain -- it returned 2!
100 of 21155
200 of 21155
300 of 21155
400 of 21155
500 of 21155
600 of 21155
700 of 21155
800 of 21155
900 of 21155
failed for 1f0cA01 for Exception:get() returned more than one domain -- it returned 2!
1000 of 21155
failed for 1f5tA02 for Exception:get() returned more than one domain -- it returned 2!
1100 of 21155
1200 of 21155
1300 of 21155
1400 of 21155
1500 of 21155
1600 of 21155
1700 of 21155
1800 of 21155
failed for 1imvA01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1imvA02 for Exception:get() returned more than one domain -- it returned 2!
1900 of 21155
2000 of 21155
failed for 1jjoC01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1jmoA01 for Exception:get() returned more than one domain -- it returned 2!
2100 of 21155
failed for 1jmoA02 for Exception:get() returned more than one domain -- it returned 2!
2200 of

16600 of 21155
16700 of 21155
16800 of 21155
16900 of 21155
17000 of 21155
17100 of 21155
failed for 3stoA01 for Exception:get() returned more than one domain -- it returned 2!
failed for 3stoA02 for Exception:get() returned more than one domain -- it returned 2!
17200 of 21155
17300 of 21155
failed for 3tmmA01 for Exception:get() returned more than one domain -- it returned 2!
failed for 3tq6B02 for Exception:get() returned more than one domain -- it returned 2!
17400 of 21155
17500 of 21155
17600 of 21155
17700 of 21155
17800 of 21155
17900 of 21155
18000 of 21155
18100 of 21155
failed for 4afxA02 for Exception:get() returned more than one domain -- it returned 2!
18200 of 21155
18300 of 21155
18400 of 21155
18500 of 21155
18600 of 21155
failed for 4dteA02 for Exception:get() returned more than one domain -- it returned 2!
18700 of 21155
failed for 4dy0B01 for Exception:get() returned more than one domain -- it returned 2!
18800 of 21155
18900 of 21155
19000 of 21155
19100 of 21155
1

In [27]:
lst = domain.objects.all()
c =  counter(lst)
p_quickheader = re.compile('|([0-9,a-z,A-Z]*)/')
for j in lst:
    c.count()
    try:
        dom1 = j.domain_id
        dom2 = seqheader_parse_cath(p_header.findall(j.classification.hmmprofile_set.all()[0].text)[0])["acc"]
        assert dom1 == dom2,"Domain %s conflict with HMM profile %s" % (dom1,dom2 )
    except Exception as e:
        c.fail(e)
        
    

0 of 34193
100 of 34193
200 of 34193
300 of 34193
400 of 34193
500 of 34193
600 of 34193
700 of 34193
800 of 34193
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
900 of 34193
1000 of 34193
1100 of 34193
1200 of 34193
1300 of 34193
1400 of 34193
1500 of 34193
1600 of 34193
1700 of 34193
1800 of 34193
1900 of 34193
2000 of 34193
2100 of 34193
2200 of 34193
2300 of 34193
2400 of 34193
2500 of 34193
2600 of 34193
2700 of 34193
2800 of 34193
list index out of range
2900 of 34193
3000 of 34193
3100 of 34193
3200 of 34193
3300 of 34193
3400 of 34193
3500 of 34193
3600 of 34193
3700 of 34193
3800 of 34193
3900 of 34193
4000 of 34193
4100 of 34193
4200 of 34193
4300 of 34193
4400 of 34193
4500 of 34193
4600 of 34193
4700 of 34193
4800

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
21700 of 34193
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
l

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
22100 of 34193
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
l

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

...

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

## HMMsearch all the loaded HMM profiles (against CATHS100)

In [53]:
%load_ext autoreload
%autoreload 0

In [54]:
from time import time
from tst.domutil.util import *
import re
import gc
# p_header = re.compile("NAME.*?\n")
# from copy import copy
# hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))
seqDB_curr = seqDB.objects.get(name = 'CATH')
seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"
behave = "HMMsearching profiles"

HMMprofile.objects.all()

t0 = time()

qset = HMMprofile.objects.order_by("id")
c = counter(range(qset.count()), per = 100)
batches = batch_qs(qset, batch_size =500)
# it = lst.iterator()
failcount = 0

# print "hi"
def worker():
    q_hits = hmmsearch(hmm,
    seqDB_curr = seqDB_curr,
    seqDB_file = seqDB_file,
    tmpdir = "/tmp/hmmsearch"                
    )

    oldhits = hmm.hit4hmm2hsp_set.all()
    for hit in q_hits:
        hsp = hit[0] ### Assume only one dom per hit
        jdict = hsp2jdict(hsp, query = hmm )
        jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"] ).id

        if not oldhits.filter(**jdict).exists():
            hit_db = hit4hmm2hsp(**jdict)
            hit_db.save()

transaction.set_autocommit( False)
if 1:
# with transaction.atomic():
    for batch in batches:
        for hmm in batch:
            c.count()
#             if c.i > 500:
#                 continue
            try:
                worker()
#                 hits = hmmsearch(hmm,
#                 seqDB_curr = seqDB_curr,
#                 seqDB_file = seqDB_file,
#                 )
            except Exception as e:
                c.fail("failed for %s for %s" % (hmm,e),)
#                 failcount += 1
            if not c.i % c.per:
                transaction.commit()



print "finshed %s from %s" % (behave, hmmlib_file)
print '%d instances of %d failed' % (c.f,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = c.f/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate
transaction.set_autocommit(True)


0 of 21078


KeyboardInterrupt: 

## Load HMM search results from flatfile (against CATHS100)

In [28]:
%load_ext autoreload
%autoreload 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print q.get()    # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


In [8]:
categories = seqDB.objects.bulk_create([
    seqDB(name = 'test', version='0'),
    seqDB(name = 'test', version='1'),
    seqDB(name = 'test', version='2'),
])


In [17]:
import sys
import multiprocessing as mp

def db_listener( q, c1):
    with transaction.atomic():
        
        ### initialise internal counter
        c2 = counter([],INF=1)
        
        while 1:
            (jdict, hmm, status) = q.get()
            if status == -1:
                break
            if hmm_income: ### When a new hmm comes
                c1.count()
                #### Check the incoming event type
                
                if isinstance(hmm_income, list): 
                    e,name = hmm_income
                    msg = "%s failed for Exception: %s " % (name,e)
                    c1.fail( msg, name)
                else:
                    if c2.i:
                        msg = " %d of %d for instances for HMM %s failed. Last exception is %s" % (c2.f, c2.i, hmm, c2.e) 
                        c1.fail( msg, hmm)
                    hmm = hmm_income
                    oldhits = hmm_income.hits
                    c2 = counter([],INF=1)  ### refresh internal counter

            else:
                if jdict == 1:
                    c2.fail('',None)
                try:
                    if not oldhits.filter(**jdict).exists():
                        hit_db = hit4hmm2hsp(**jdict)
                        hit_db.save()
                except Exception as e:
                    c2.fail('', None )
                finally:
                    c2.count()
                    
        return (c)
    

def parse_worker( q, q_hits, domain2hmm):
# def parse_worker( q_hits, ):
#     global domain2hmm
#     jdict = {'acc_avg': 0.99,
#      'bias': 1.8,
#      'bitscore': 114.9,
#      'end': 81,
#      'logCevalue': -35.886056647693167,
#      'logIevalue': -32.086186147616282,
#      'query_id': 18006L,
#      'start': 0,
#      'target_id': '1rl6A01'}
    def acc_mapper(hit):
        hit.id = seqheader_parse_cath(hit.id)["acc"]
        return hit    
    
    acc = p_cathdomain.findall(q_hits.id)[0]
    status = 0
    try:
        hmm = HMMprofile.objects.get(pk=domain2hmm[acc])
        
        q.put([None,hmm,status])
        
        q_hits = q_hits.hit_map(acc_mapper) 
        for hit in q_hits:
            try:
                hsp = hit[0] ### Assume only one dom per hit
                jdict = hsp2jdict(hsp, query = hmm )
                jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"] ).id
                q.put([jdict,None,0])
            except:
                q.put([1,None,0])

                  
    
    except Exception as e:
#         hmm_out = [e,hmm]
        q.put([None,[e,acc],0] )
        
    finally:
        return 0
    
#     q.put([jdict, hmm_out, status])
#     q.id 
    


In [87]:
import multiprocessing as mp
m = mp.Manager()
sdct  = m.dict(domain2hmm)


In [50]:
from tst.domutil.util import *
def worker():
    q_hits = hmmsearch(hmm,
    seqDB_curr = seqDB_curr,
    seqDB_file = seqDB_file,
    tmpdir = "/tmp/hmmsearch"                
    )
    return q_hits
seqDB_curr = seqDB.objects.get(name='CATH')
seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"
q_hits = worker()

In [ ]:
# for i in pool.imap_unordered(f, range(10)):
# print i


watcher = pool.apply_async( db_listener, (q, c));
jobs = [];
for q_hits in parser:
    job = pool.apply_async( parse_worker, (q, q_hits) )
    jobs.append( job )
for job in jobs:
    job.get()
q.put([None,None,-1])
[status, c] = watcher.get()
pool.close()

In [99]:
%autoreload 0


In [2]:
from tst.domutil.util import *
counter([])

/export/people/zcbtfge/Documents/repos/test_envs/django_CATH/local/lib/python2.7/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


<tst.domutil.util.counter instance at 0x7fd4416d7b48>

In [11]:
import re
p_cathdomain = re.compile("([0-9,a-z,A-Z]{7})")

In [18]:
from time import time
from tst.domutil.util import *
import re
import gc
# p_header = re.compile("NAME.*?\n")
# from copy import copy
# hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))
seqDB_curr = seqDB.objects.get(name = 'CATH')
seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"
behave = "HMMsearching profiles"

HMMprofile.objects.all()
INPUT = full("$SEQlib/hmm/CATHS100.domtbl")
parser = SearchIO.parse( INPUT, 'hmmsearch3-domtab' )


t0 = time()
qset = HMMprofile.objects.order_by("id")
c = counter([], INF = 1, per = 10)
batches = batch_qs(qset, batch_size =500)
# it = lst.iterator()



if __name__=='__main__':
    pool = mp.Pool( mp.cpu_count() - 1)
    manager = mp.Manager()
#     manager.start()
    q = manager.Queue();                      
    
    domain2hmm  = dict(HMMprofile.objects.defer('text').values_list("cath_node__domain__domain_id","id"))
#     sdct  = manager.dict(domain2hmm)
    domain2hmm = manager.dict(domain2hmm)
    Global = manager.Namespace()
    Global.p_cathdomain = re.compile("([0-9,a-z,A-Z]{7})")
    
    watcher = pool.apply_async( db_listener, (q, c));
    jobs = [];
#     def f(it):
#         return (it)
#     parser = pool.imap(, parser)
    c0 = counter([],INF=1)
    for q_hits in parser:
        c0.count()
        job = pool.apply_async( parse_worker, (q, q_hits, domain2hmm) )
#     for q_hits in parser:
#         job = pool.apply_async( parse_worker, (q, q_hits, sdct) )
#         jobs.append( job )
#     for job in jobs:
#         job.get()
    q.put([None,None,-1])
#     (status, c) = watcher.get()
    c = watcher.get()
    pool.close()

# c.finish()
print "finshed %s from %s" % (behave, hmmlib_file)
print '%d instances of %d failed' % (c.f,c.i)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = c.f/float(c.i) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate


0 of -1
100 of -1
200 of -1
300 of -1
400 of -1
500 of -1
600 of -1
700 of -1
800 of -1
900 of -1
1000 of -1
1100 of -1
1200 of -1
1300 of -1
1400 of -1
1500 of -1
1600 of -1
1700 of -1
1800 of -1
1900 of -1
2000 of -1
2100 of -1
2200 of -1


Process PoolWorker-43:
Process PoolWorker-42:
Traceback (most recent call last):
Process PoolWorker-45:
Process PoolWorker-44:
Process PoolWorker-41:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-46:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self.run()
Process PoolWorker-40:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
Traceback (most recent call last):
  File "/usr/lib/p

KeyboardInterrupt: 

  File "/usr/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    task = get()
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/usr/lib/python2.7/multiprocessing/queues.py", line 378, in get
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    task = get()
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
    task = get()
  File "/usr/lib/python2.7/multiprocessing/queues.py", line 376, in get
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
  File "/usr/lib/python2.7/multiprocessing/queues.py", line 376, in get
  File "/usr/lib/pyth

In [28]:
from tst.domutil.util import *
INPUT = full('$SEQlib/hmm/CATHS100.domtbl')
with open(INPUT,'r') as f:    
    g = f.readlines()
def f(line):
    return SearchIO.read(line,'hmmsearch3-domtab')
pool = mp.Pool(7)
hi = pool.imap(f,g)

In [29]:
next(hi)

IOError: [Errno 2] No such file or directory: '#                                                                                            --- full sequence --- -------------- this domain -------------   hmm coord   ali coord   env coord\n'

In [24]:
idx_file  = SearchIO.index( INPUT, 'hmmsearch3-domtab' )
# = 


IndexError: list index out of range

In [22]:
parser = SearchIO.parse( INPUT, 'hmmsearch3-domtab' )
c0 = counter([],INF=1)
for q_hits in parser:
#     print q_hits
    c0.count()

0 of -1
100 of -1
200 of -1
300 of -1
400 of -1
500 of -1
600 of -1
700 of -1
800 of -1
900 of -1
1000 of -1
1100 of -1
1200 of -1
1300 of -1


KeyboardInterrupt: 

In [21]:
?Global

In [14]:
print next(r)


NameError: global name 'Global' is not defined

In [31]:
from time import time
from tst.domutil.util import *
import re
import gc
# p_header = re.compile("NAME.*?\n")
# from copy import copy
# hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))
seqDB_curr = seqDB.objects.get(name = 'CATH')
seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"
behave = "HMMsearching profiles"

HMMprofile.objects.all()

t0 = time()

qset = HMMprofile.objects.order_by("id")
c = counter(range(qset.count()), per = 100)
batches = batch_qs(qset, batch_size =500)
# it = lst.iterator()
failcount = 0

# print "hi"
def worker():
    q_hits = hmmsearch(hmm,
    seqDB_curr = seqDB_curr,
    seqDB_file = seqDB_file,
    tmpdir = "/tmp/hmmsearch"                
    )

    oldhits = hmm.hit4hmm2hsp_set.all()
    for hit in q_hits:
        hsp = hit[0] ### Assume only one dom per hit
        jdict = hsp2jdict(hsp, query = hmm )
        jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"]).id

        if not oldhits.filter(**jdict).exists():
            hit_db = hit4hmm2hsp(**jdict)
            hit_db.save()

transaction.set_autocommit( False)
if 1:
# with transaction.atomic():
    for batch in batches:
        for hmm in batch:
            c.count()
#             if c.i > 500:
#                 continue
            try:
                worker()
#                 hits = hmmsearch(hmm,
#                 seqDB_curr = seqDB_curr,
#                 seqDB_file = seqDB_file,
#                 )
            except Exception as e:
                c.fail("failed for %s for %s" % (hmm,e),)
#                 failcount += 1
            if not c.i % c.per:
                transaction.commit()



print "finshed %s from %s" % (behave, hmmlib_file)
print '%d instances of %d failed' % (c.f,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = c.f/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate
transaction.set_autocommit(True)


0 of 21086
100 of 21086
200 of 21086
300 of 21086
400 of 21086
500 of 21086
600 of 21086
failed for HMM for 3.90.226.10.3.0  for get() returned more than one sequence -- it returned 2!
700 of 21086
800 of 21086


tst/domutil/util.py:319: RuntimeWarning: divide by zero encountered in log10
  jdict["logCevalue"] = np.log10(jdict.pop("evalue_cond"))
tst/domutil/util.py:320: RuntimeWarning: divide by zero encountered in log10
  jdict["logIevalue"] = np.log10(jdict.pop("evalue"))


failed for HMM for 3.20.20.350.1.0  for (1054, "Unknown column 'inf' in 'where clause'")
failed for HMM for 3.90.226.10.46.0  for get() returned more than one sequence -- it returned 2!
900 of 21086
1000 of 21086
1100 of 21086
failed for HMM for 3.60.20.10.29.0  for get() returned more than one sequence -- it returned 2!
1200 of 21086
1300 of 21086
1400 of 21086
failed for HMM for 3.20.70.20.1.0  for (1054, "Unknown column 'inf' in 'where clause'")
1500 of 21086
failed for HMM for 3.90.226.10.62.0  for get() returned more than one sequence -- it returned 2!
1600 of 21086
1700 of 21086
1800 of 21086
1900 of 21086
failed for HMM for 1.20.1130.10.1.0  for (1054, "Unknown column 'inf' in 'where clause'")
2000 of 21086
2100 of 21086
2200 of 21086
2300 of 21086
2400 of 21086
2500 of 21086
2600 of 21086
failed for HMM for 3.90.1290.10.2.0  for get() returned more than one sequence -- it returned 2!
2700 of 21086
2800 of 21086
2900 of 21086
3000 of 21086
3100 of 21086
3200 of 21086
failed for 

14500 of 21086
14600 of 21086
failed for HMM for 3.90.226.10.35.0  for get() returned more than one sequence -- it returned 2!
14700 of 21086
14800 of 21086
14900 of 21086
15000 of 21086
failed for HMM for 3.90.226.10.29.0  for get() returned more than one sequence -- it returned 2!
15100 of 21086
15200 of 21086
failed for HMM for 3.90.226.10.43.0  for get() returned more than one sequence -- it returned 2!
15300 of 21086
failed for HMM for 3.30.70.100.47.0  for get() returned more than one sequence -- it returned 2!
15400 of 21086
failed for HMM for 3.90.226.10.15.0  for get() returned more than one sequence -- it returned 2!
15500 of 21086
failed for HMM for 3.60.20.10.17.0  for get() returned more than one sequence -- it returned 2!
failed for HMM for 3.60.20.10.19.0  for get() returned more than one sequence -- it returned 2!
15600 of 21086
failed for HMM for 3.90.226.10.8.0  for get() returned more than one sequence -- it returned 2!
failed for HMM for 3.90.226.10.11.0  for get() 

In [ ]:
def worker():
    q_hits = hmmsearch(hmm,
    seqDB_curr = seqDB_curr,
    seqDB_file = seqDB_file,
    tmpdir = "/tmp/hmmsearch"                
    )

In [37]:
def worker():
    q_hits = hmmsearch(hmm,
    seqDB_curr = seqDB_curr,
    seqDB_file = seqDB_file,
    tmpdir = "/tmp/hmmsearch"                
    )
    return q_hits

In [1]:
r =  classification.objects.get_bytree("3.30.497.10")


In [46]:
hit = q_hits[0]
hsp = hit[0]
jdict = hsp2jdict(hsp, hmm)

In [61]:
p_cathdomain = re.compile("([0-9,a-z,A-Z]{7})")

In [63]:
p_cathdomain.findall(q_hits.id)[0]

'4a17E01'

In [76]:
domain2hmm  = dict(HMMprofile.objects.defer('text').values_list("cath_node__domain__domain_id","id"))


The slowest run took 6.58 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 181 ns per loop


In [88]:
%timeit d['4a17E01']

The slowest run took 1244.44 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 23.5 µs per loop


In [81]:
# len(domain2hmm)
from collections import OrderedDict
d = dict(HMMprofile.objects.defer('text').values_list("cath_node__domain__domain_id","id"))
domain2hmm = OrderedDict(sorted(domain2hmm.items(), key=lambda t: t[0]))
%timeit domain2hmm['4a17E01']

The slowest run took 11.65 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 266 ns per loop


In [74]:
domain2hmm['4a17E01']

18006L

In [47]:
jdict

{'acc_avg': 0.99,
 'bias': 1.8,
 'bitscore': 114.9,
 'end': 81,
 'logCevalue': -35.886056647693167,
 'logIevalue': -32.086186147616282,
 'query_id': 18006L,
 'start': 0,
 'target_id': '1rl6A01'}

In [29]:
r =  classification.objects.get(Class = 3,
                                arch = 90,
                                topo = 930,
                                homsf = 12,
                               s35 = 7,
                               s60 = 0,).hmmprofile_set.all()
hmm = r[0]

In [22]:
print r.parent.parent.parent

3.0


In [23]:
r =  classification.objects.get_bytree("3")

print r
# print classification.objects.count()
# classification.objects

(None, 0)


In [24]:
hmm = classification.objects.get_bytree("3.90.930.12.7.0")[0].hmmprofile_set.all()

AttributeError: 'NoneType' object has no attribute 'hmmprofile_set'

In [ ]:
fixhmms = HMMprofile.objects.filter(hits__isnull = True)
fixhmms_ids = 

In [65]:
max(jdict['logCevalue'],-1000)
# hsp.__dict__

-1000

In [73]:
# fixhmms.count()
# len(fixhmms)
for hmm in fixhmms:
#     worker()
    q_hits = hmmsearch(hmm,
    seqDB_curr = seqDB_curr,
    seqDB_file = seqDB_file,
    tmpdir = "/tmp/hmmsearch"                
    )
    oldhits = hmm.hit4hmm2hsp_set.all()
    for hit in q_hits:
        hsp = hit[0] ### Assume only one dom per hit
        jdict = hsp2jdict(hsp[:], query = hmm )
        jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"]).id

        if not oldhits.filter(**jdict).exists():
            hit_db = hit4hmm2hsp(**jdict)
            hit_db.save()
#     try:
#         worker()
#     except Exception as e:
#         print e

/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log10
  # This is added back by InteractiveShellApp.init_path()
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


DoesNotExist: sequence matching query does not exist.

In [44]:
sDB = seqDB.objects.get(name = 'CATH')
sDB.sequence_set.count()

80422

In [42]:
sequence.objects.values_list("acc").distinct().count()

80403

In [39]:
cnode, code = classification.objects.get_bytree("3.90.930.12.7.0")
if code:
    print cnode
#     print cnode.hmmprofile_set.all()
    hmm = cnode.hmmprofile_set.first()
# batch_qs
# lst.order_by("id")

3.90.930.12.7.0
<QuerySet [<HMMprofile: HMM for 3.90.930.12.7.0 >]>


In [18]:
# %load_ext reload
# del tst.domutil.util
%autoreload 2


In [ ]:
parser = SearchIO.parse( INPUT, tbl_format )
c = counter(parser)


In [ ]:
# parser = SearchIO.parse( INPUT, tbl_format )


In [5]:
## Load HMMsearch results from an external file

from time import time
from tst.domutil.util import *
from tst.utils_db import *
from Bio import SearchIO
# p_header = re.compile("NAME.*?\n")
# from copy import copy
# hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))


seqDB_curr = seqDB.objects.get(name = 'CATH')
# seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"
INPUT = "/media/shouldsee/DATA_FENG/data/archive/CATHS100_v410.domtbl"
tbl_format = 'hmmsearch3-domtab'
hmm_resultfile = INPUT
behave = "Loading HMMsearch results"
t0 = time()

parser = SearchIO.parse( INPUT, tbl_format )
c = counter(parser, INF = True)
parser = SearchIO.parse( INPUT, tbl_format )

# if 1:
with transaction.atomic():
    for q_hits in parser:
#         q_hits = next(parser)
    #     q_hits = next(parser)    
        try:
#         if 1:
            q_hits.id = seqheader_parse_cath(q_hits.id)["acc"]
            hmm = domain.objects.get(domain_id = q_hits.id).classification.hmmprofile_set.first()
            q_hits = q_hits.hit_map(acc_mapper_cath)

            oldhits = hmm.hit4hmm2hsp_set.all()
            for hit in q_hits:
                hsp = hit[0] ### Assume only one dom per hit
                jdict = hsp2jdict(hsp, query = hmm )
                jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"]).id
                
                if not oldhits.filter(**jdict).exists():
                    hit_db = hit4hmm2hsp(**jdict)
                    hit_db.save()
#         try:
#             pass
        except Exception as e:
            c.fail("Failed at %s for Exception:%s" % (hmm, e))
        finally:
            c.count()
    
            
c.finish()

print "finshed %s from %s" % (behave, INPUT)
print '%d instances of %d failed' % (c.f,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = c.f/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate


0 of -1


KeyboardInterrupt: 

In [45]:
print "%d" % None

TypeError: %d format: a number is required, not NoneType

In [12]:
hmmpf_file = full("$SEQlib/cath-S35-hmm3.lib")
# hmmpf_file = full("$SEQlib/tmp.hmmpf")

def search_idx(q_acc, fname, acc_func):
    with open(fname, 'r') as f:
        line = f.readline()
        idx = 0
        while line:
#             header = line.rstrip("\n")[6:]
#             acc = seqheader_parse_cath(header)["acc"]
            ac = acc_func(line)
            if acc == q_acc:
                break
            
            line = f.readline()
            idx += 1
        if not line:
            return None
        else:
            return idx
        
        

        
import subprocess
def search_lib( q_acc, fname = hmmpf_file, acc_func = None):
    
    ##### check whether an index file exists *.idx
    idxfile = fname + '.idx'
    if not os.path.isfile(idxfile):
        subprocess.call(['cat',fname,'|','grep','NAME','>',idxfile])
    if not acc_func:
        acc_func = lambda line: seqheader_parse_cath(line.rstrip("\n")[6:])["acc"]
    
    idx = search_idx(q_acc ,idxfile, acc_func)
    print idx

    if not idx:
        return None
    else:
        with open(fname,'r') as f:
            line = 1
            cidx = 0
#             ln = 0
            buf = ""
            while line:
                line = f.readline()                
                if cidx == idx:
                    buf += line
                if line.rstrip('\n')=='//':
                    cidx += 1
                    if buf:
                        return buf + line


# p_acc = "2y8gB00"
# dom = domain.objects.get(domain_id = p_acc)
# print search_lib( p_acc )        
# subprocess.call(['hmmscan', '--domtblout', 'output.txt', '/location/to/profile.hmm', 'input.proteins.fasta']

### import scanning resultv
                

In [72]:
def hsp2jdict(hsp,query = None):
    jdict = hsp.__dict__
    jdict["query_id"] = query.id
#     print hsp.hit_id
    jdict["target_id"] = hsp.hit_id
    # jdict["target_id"] = sequence.objects.get( acc = hsp.hit_id).id
    it = jdict.pop('_items')
    jdict.pop('domain_index')
    jdict["start"]=jdict.pop("env_start")
    jdict["end"]=jdict.pop("env_end")
    jdict["logCevalue"] = max(-1000,np.log10(jdict.pop("evalue_cond")))
    jdict["logIevalue"] = max(-1000,np.log10(jdict.pop("evalue")))
    
    return jdict


def hmmsearch(hmm,seqDB_curr = None, seqDB_file = None, tmpdir = "/tmp", 
	tbl_format = 'hmmsearch3-domtab'):
#     if ["seqDB_curr"] in locals().keys():
    if not seqDB_curr:
        seqDB_curr = seqDB.objects.get(name = 'CATH')
#         seqDB_file = seqDB_curr.filepath
    if seqDB_file:
        seqDB_file = full(seqDB_file)
    

    tmphmm = "%s/hmm" % tmpdir
    with open(tmphmm,'w') as f:
        f.write(hmm.text)
    cmd = ["hmmsearch","--noali","-o",tmpdir+"/log","--domtblout",tmpdir + "/domtbl", 
                                     tmphmm, seqDB_file,]
    # print ' '.join(cmd)
    try:
        qtext = subprocess.check_output( cmd )
    except Exception as e:
    	print 'CMD is: %s' % (' '.join(cmd))
        raise e 
    # del qtext
    # gc.collect()

    parser = SearchIO.parse( tmpdir + "/domtbl", tbl_format )
    q_hits = next(parser)
    q_hits.id = seqheader_parse_cath(q_hits.id)["acc"]
    def acc_mapper(hit):
#         hit.query
        hit.id = seqheader_parse_cath(hit.id)["acc"]
        return hit
    q_hits = q_hits.hit_map(acc_mapper)
    return q_hits
    
	# oldhits = hmm.hit4hmm2hsp_set.all()
	# for hit in q_hits:
	#     hsp = hit[0] ### Assume only one dom per hit
	#     jdict = hsp2jdict(hsp, query = hmm )
	#     jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"]).id
	#     if not oldhits.filter(**jdict).exists():
	#         hit_db = hit4hmm2hsp(**jdict)
	#         hit_db.save()

#     dis_q = hmm.hit4hmm2hsp_set.values_list("target",flat = True).distinct()
    
#     q = hmm.hit4hmm2hsp_set.all()

#     if dis_q.count() < q.count():
#         q = q.exclude(id__in=list(dis_q.values_list("id",flat = True)) )
# #         q.delete()
# #         dis_q.update()
        
    return hmm.hit4hmm2hsp_set

In [70]:
1E-300

1e-300

In [158]:
s35A = s35( "2q09A01", hmmlib_file = hmmpf_file)
s35B = s35( "8ohmA03", hmmlib_file = hmmpf_file)
print "finished"

with transaction.atomic():
    s35A.hmmsearch(
        seqDB_curr = seqDB.objects.get(name = 'CATH'),
        seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa",
    )
print s35A.hitset.count()

with transaction.atomic():
    s35B.hmmsearch(
        seqDB_curr = seqDB.objects.get(name = 'CATH'),
        seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa",
    )
print s35B.hitset.count()

print hit4hmm2hsp.objects.count()

10234
21154
finished
